## **Cohere Reranker**

Cohere에서 제공하는 Reranker API

### **Cohere 다국어 지원 모델**

- Embedding: `embed-multilingual-v3.0`, `embed-multilingual-light-v3.0`, `embed-multilingual-v2.0`
- Reranker: `rerank-multilingual-v3.0`, `rerank-multilingual-v2.0`

In [ ]:
# 설치
# !pip install -qU cohere

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 임베딩 모델 정의
embeddings = CohereEmbeddings(model="embed-multilingual-v3.0")

# 검색기 초기화
retriever = FAISS.from_documents(
    documents= texts, 
    embedding= embeddings
).as_retriever(search_kwargs={"k": 10})

# 질의문
query = "Word2Vec 에 대해서 알려줘!"

# 문서 검색
docs = retriever.invoke(query)

### **Reranker 정의**

In [2]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

# 문서 재정렬 모델 설정
compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n = 5)

# 문맥 압축 검색기 설정
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("Word2Vec 에 대해서 알려줘!")

compressed_docs[0].page_content

'Crawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec\n\n정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)'